**Niks Pille**

All Tickers

In [9]:
#Getting all tickers in a list
All_Tickers = []
for i in range(4521):
    All_Tickers.append(Only_Common_Stocks['symbol'][i])

Creating 'Stock_With_All_Data'

In [171]:
Stocks_With_All_Data = []
for i in range(4521):
    if len(yf.Ticker(All_Tickers[i]).history(start='2000-01-02', end='2020-01-01')[['Close']]) == 5031:
        Stocks_With_All_Data.append(All_Tickers[i])

- AGM.A: No data found, symbol may be delisted
- ALUS: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- AMHC: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- ARQT: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BATL: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BDTX: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BEAM: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BF.A: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- BH.A: No data found, symbol may be delisted
- BIO.B: No data found for this date range, symbol may be delisted
- BRK.A: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BSBK: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BWL.A: No data found, symbol may be delisted
- CHPM: Data doesn't exist for startDate = 946767600, endDate 

In [186]:
with open('Stocks_With_Data.csv','w',newline='') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    wr.writerow(Stocks_With_All_Data)

Creating 'Three_Month_Treasury_Bill'

In [38]:
#DTB3 is taken from https://fred.stlouisfed.org/
DTB3 = pd.read_csv('DTB3.csv', index_col = 0)
#Using Apple as a benchmark
aapl = yf.Ticker('aapl').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Getting dates from Apple
Dates = []
for i in range(len(aapl)):
    if str(aapl.index[i])[:10] in DTB3.index:
        Dates.append(str(aapl.index[i])[:10])
#Sorting out missing values
Three_Month_Treasury_Bill = DTB3.loc[Dates][DTB3.loc[Dates].DTB3 != '.']
#Creating a csv-file with the Three Month Treasury Bill
Three_Month_Treasury_Bill.to_csv('Three_Month_Treasury_Bill.csv')

Creating 'All_Stocks'

In [45]:
#Make a table with the new dates
Ny = DTB3.loc[Dates]
#Make a new column with strings
Ny['Datoer'] = Dates
#Set the new column as index to make it a callable string
Nyy = Ny.set_index('Datoer')
#A for loop to get all tables for the chosen stocks into a single list, to be used in concatenating
stock = []
for i in list(dict.fromkeys(Stocks)):
    asset = yf.Ticker(i).history(start='2000-01-02', end='2020-01-01')[['Close']]
    asset.columns = [i]
    stock.append(asset)
#Concat all the tables
Concat_Table = pd.concat([i for i in stock], axis=1, join='inner')
#Concat the stocks with the risk free rate
All_Tables = pd.concat([Nyy, Concat_Table.reindex(Nyy.index)],axis=1)
#Save to a csv file
All_Tables.to_csv('All_Stocks.csv')

Creating 'Table'

In [48]:
Everything = pd.read_csv('All_Stocks.csv', index_col=0)
#Russell3000 is used as the market rate since it covers 98% of the American market
Russell_3000 = yf.Ticker('^RUA').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Sorting out missing values from the risk-free rate
gj = Everything.loc[:][Everything.loc[:].DTB3 != '.']
#Making a new date list with the dates from Russell 3000
Datess = []
for i in range(len(Russell_3000)):
    if str(Russell_3000.index[i])[:10] in gj.index:
        Datess.append(str(Russell_3000.index[i])[:10])
#Making the dates strings so they are callable
gh = []
for i in range(len(Russell_3000)):
    gh.append(str(Russell_3000.index[i])[:10])
#Making a new column with the string dates and set it as index
Russell_3000['Datoer'] = gh
gk = Russell_3000.set_index('Datoer')
#Set the name of the column
gk.columns = ['Russell3000']
#Concat Russell3000 to the rest
Table = pd.concat([gj.loc[Datess],gk.loc[Datess]],axis=1)
Table.to_csv('Table.csv')

# Bachelorproject

Documentation:  
https://addisonlynch.github.io/iexfinance/stable/stocks.html  
https://github.com/ranaroussi/yfinance

## Getting Data

In [1]:
#All packages used
from iexfinance.stocks import Stock
from iexfinance.refdata import get_symbols
import yfinance as yf
import numpy as np
import pandas as pd
import csv
token = 'sk_32c5d1b5f6b044f294391ae88eac3e47'

In [2]:
#Get a table with everything on the financial markets in USA
Alle_symboler = get_symbols(output_format='pandas', token=token)
#Sorting out all the non common stocks
Only_Common_Stocks = get_symbols()[get_symbols().type == 'cs'].reset_index(drop=True)

In [2]:
with open('Stocks_With_Data.csv',newline='') as file:
    reader = csv.reader(file)
    Stocks_With_All_Data = list(reader)
#Drawing 300 random stocks for which to be analyzed
Stocks = []
np.random.seed(2020)
for i in range(300):
    Stocks.append(Stocks_With_All_Data[0][np.random.randint(0,len(Stocks_With_All_Data[0]))])

Stocks.sort()
Every_Stock = list(dict.fromkeys(Stocks))

### Adjusting so all data match with no missing values

In [3]:
All = pd.read_csv('Table.csv', index_col=0)

In [6]:
columns = []
for i in range(283):
    columns.append(All.columns[i])
#Creating a table with the log values
Logtable = pd.DataFrame()
for i in range(283):
    Logtable[columns[i]] = All[columns[i]].apply(np.log)

In [9]:
Daily = pd.DataFrame()
for i in range(len(columns)):
    Daily[columns[i]] = np.diff(Logtable[columns[i]],axis=0)

In [61]:
Daily_returns = Daily.set_index(Logtable.index.values[0:-1])
a = ['NaN','inf', '-inf']
Daily_returnss = Daily_returns[~Daily_returns['DTB3'].isin(a)]
Daily_returnss.to_csv('Daily_returns.csv')

In [62]:
Daily_returns = pd.read_csv('Daily_returns.csv', index_col=0)

In [63]:
Excess_return = pd.DataFrame()

for i in range(len(columns)):
    Excess_return[columns[i]] = Daily_returns[columns[i]] - Daily_returns['DTB3']
del Excess_return['DTB3']

In [64]:
Excess_return.to_csv('Excess_return.csv')